# MLB Team Win Total Model — End-to-End Notebook

This single notebook reproduces the workflow we ran in-chat:

1. Load historical team data (FanGraphs) and normalize multi-year totals to per-season
2. Train a ridge regression model using the same feature set as your `input.xlsx`
3. Validate with out-of-sample cross-validation (MAE, RMSE, R²)
4. Load your `input.xlsx` (team-level inputs aggregated from player projections) and generate projected wins
5. (Optional) Compare projections to sportsbook win totals and plot a scatter chart

> **Files expected in the same environment**  
- Historical data: `/mnt/data/fangraphs-leaderboards(21).csv`  
- Team inputs: `/mnt/data/input.xlsx`


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


## 1) Load historical data and prepare training set

In [ ]:
train = pd.read_csv("/mnt/data/fangraphs-leaderboards(21).csv")

# Training file contains 3-year totals; convert to per-season so the target matches your single-season inputs
train["wins_ps"] = train["wins"] / 3

totals_cols = ['G','PA','HR','R','RBI','SB','BsR','Off','Def','WAR','1B','2B','3B']
for c in totals_cols:
    if c in train.columns:
        train[c] = train[c] / 3

# Map training column names to the feature names used in input.xlsx
rename_map = {
    "WAR": "war",
    "ERA": "era",
    "WHIP": "whip",
    "K-BB%": "k-bb",
    "OBP": "obp",
    "R": "runs",
    "SLG": "slg"
}
train = train.rename(columns=rename_map)

features = ["war","era","whip","k-bb","obp","runs","slg"]
X = train[features]
y = train["wins_ps"]

X.head(), y.head()


## 2) Quick feature correlations (sanity check)

In [ ]:
corr = X.corrwith(y).sort_values(key=abs, ascending=False)
corr_df = corr.to_frame("Correlation_with_Wins")
corr_df


In [ ]:
plt.figure(figsize=(8,5))
plt.barh(corr_df.index, corr_df["Correlation_with_Wins"])
plt.axvline(0, color="black")
plt.title("Correlation of Selected Metrics with Wins")
plt.tight_layout()
plt.show()


## 3) Train + validate model (out-of-sample cross-validation)

In [ ]:
model = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("ridge", RidgeCV(alphas=np.logspace(-3,3,100)))
])

cv = KFold(n_splits=5, shuffle=True, random_state=42)
y_pred = cross_val_predict(model, X, y, cv=cv)

mae = mean_absolute_error(y, y_pred)
rmse = mean_squared_error(y, y_pred, squared=False)
r2 = r2_score(y, y_pred)

{"MAE": mae, "RMSE": rmse, "R2": r2}


In [ ]:
# Residual plot (optional diagnostic)
resid = y - y_pred

plt.figure(figsize=(7,4))
plt.scatter(y_pred, resid)
plt.axhline(0, color="black", linewidth=0.8)
plt.title("Residuals vs Predicted Wins (Cross-Validation)")
plt.xlabel("Predicted wins (per season)")
plt.ylabel("Residual (actual - predicted)")
plt.tight_layout()
plt.show()


## 4) Fit final model on all historical data

In [ ]:
model.fit(X, y)


## 5) Load your team inputs (`input.xlsx`) and predict wins

In [ ]:
proj = pd.read_excel("/mnt/data/input.xlsx")

# Expecting columns: team, war, era, whip, k-bb, obp, runs, slg
missing = [c for c in ["team"] + features if c not in proj.columns]
if missing:
    raise ValueError(f"input.xlsx is missing expected columns: {missing}")

proj["proj_wins"] = model.predict(proj[features])
proj_out = proj[["team","proj_wins"]].sort_values("proj_wins", ascending=False)
proj_out


In [ ]:
# Export model results (optional)
proj_out.to_csv("/mnt/data/model_win_projections.csv", index=False)
"/mnt/data/model_win_projections.csv"


## 6) Optional: Compare to sportsbook win totals + scatter plot

In [ ]:
# Sportsbook 2026 win totals (O/U) used in our chat comparison
sportsbook = pd.DataFrame({
    "team": ["LAD","NYY","PHI","TOR","SEA","CHC","ATL","BOS","MIL","DET","NYM","SDP","BAL","HOU","CIN","TEX","SFG","KCR","CLE","ARI","TBR","MIN","CHW","PIT","STL","MIA","WSN","ath","COL","LAA"],
    "sportsbook_ou": [99.5,93.5,92.5,91.5,90.5,88.5,87.5,87.5,87.5,86.5,86.5,85.5,84.5,84.5,83.5,83.5,82.5,81.5,80.5,78.5,78.5,75.5,74.5,71.5,71.5,70.5,69.5,68.5,54.5,62.5]
})

comp = proj.merge(sportsbook, on="team", how="inner")
if comp.empty:
    raise ValueError("No teams matched between input.xlsx and sportsbook table. Check team abbreviations.")

pearson = comp["proj_wins"].corr(comp["sportsbook_ou"], method="pearson")
spearman = comp["proj_wins"].corr(comp["sportsbook_ou"], method="spearman")

{"pearson_r": pearson, "spearman_rho": spearman}


In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(comp["sportsbook_ou"], comp["proj_wins"])

mn = min(comp["sportsbook_ou"].min(), comp["proj_wins"].min())
mx = max(comp["sportsbook_ou"].max(), comp["proj_wins"].max())
plt.plot([mn, mx], [mn, mx])

plt.xlabel("Sportsbook Win Total (O/U)")
plt.ylabel("Model Projected Wins")
plt.title("Model Projected Wins vs Sportsbook Win Totals")
plt.tight_layout()
plt.show()


In [ ]:
comp["diff_model_minus_market"] = comp["proj_wins"] - comp["sportsbook_ou"]
diff_table = comp[["team","proj_wins","sportsbook_ou","diff_model_minus_market"]].sort_values(
    "diff_model_minus_market", key=lambda s: s.abs(), ascending=False
)
diff_table


In [ ]:
# Export market comparison (optional)
diff_table.to_csv("/mnt/data/model_vs_sportsbook.csv", index=False)
"/mnt/data/model_vs_sportsbook.csv"
